In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys
import glob
import numpy as np
import pandas as pd
import subprocess
from praatio import textgrid as tgio
import argparse

sys.path.append('../utils/')

from config import *
import dataset_utils as utils
from preproc_utils import cut_stimulus_segments, get_cut_times

In [34]:
experiment_version = 'final-multimodal-01'
task = 'howtodraw'
stim_type = 'video'
subject = 'sub-00001'

# set directories
stim_dir = os.path.join(BASE_DIR, 'stimuli')
presentation_orders_dir = os.path.join(stim_dir, 'presentation_orders', experiment_version, task)

if stim_type == 'video':
    stim_fn = glob.glob(os.path.join(stim_dir, 'video', f'{task}*.mp4'))
    jspsych_out_dir = os.path.join(presentation_orders_dir, 'jspsych-video')
else:
    stim_fn = glob.glob(os.path.join(stim_dir, 'audio', f'{task}*.wav'))
    jspsych_out_dir = os.path.join(presentation_orders_dir, 'jspsych')

In [35]:

# load preprocessed transcript and find indices that are to be predicted
df_preproc_fn = os.path.join(presentation_orders_dir, 'preproc', f'{subject}_task-{task}.csv')
df_preproc = pd.read_csv(df_preproc_fn)

In [78]:

# Create sequential pairs
candidate_idxs = np.where(df_preproc['NWP_Candidate'].to_numpy())[0] # First get indices
candidate_idxs = np.concatenate([[0], candidate_idxs], axis=0) # Add the first item for the first cut
segments = np.vstack((candidate_idxs[:-1], candidate_idxs[1:]-1)).T # Stack and make pairs
segment_indices = segments.tolist()


In [86]:
print (onset)

1.840572650987489


In [84]:
from tqdm import tqdm

for i, (start_idx, end_idx) in enumerate(tqdm(segment_indices, desc="Cutting stimulus segments")):
    onset, offset, duration = get_cut_times(df_preproc, start_idx, end_idx)

    sys.exit(0)
    print (onset)

Cutting stimulus segments:   0%|          | 0/81 [00:00<?, ?it/s]


SystemExit: 0

In [79]:
segment_indices

[[0, 20],
 [21, 59],
 [60, 121],
 [122, 141],
 [142, 178],
 [179, 207],
 [208, 287],
 [288, 298],
 [299, 332],
 [333, 343],
 [344, 378],
 [379, 403],
 [404, 416],
 [417, 429],
 [430, 454],
 [455, 484],
 [485, 497],
 [498, 509],
 [510, 532],
 [533, 570],
 [571, 583],
 [584, 681],
 [682, 696],
 [697, 717],
 [718, 733],
 [734, 752],
 [753, 793],
 [794, 825],
 [826, 837],
 [838, 849],
 [850, 864],
 [865, 876],
 [877, 887],
 [888, 909],
 [910, 924],
 [925, 952],
 [953, 971],
 [972, 1011],
 [1012, 1024],
 [1025, 1039],
 [1040, 1055],
 [1056, 1064],
 [1065, 1069],
 [1070, 1075],
 [1076, 1081],
 [1082, 1093],
 [1094, 1100],
 [1101, 1106],
 [1107, 1153],
 [1154, 1175],
 [1176, 1187],
 [1188, 1228],
 [1229, 1262],
 [1263, 1283],
 [1284, 1295],
 [1296, 1308],
 [1309, 1321],
 [1322, 1339],
 [1340, 1379],
 [1380, 1410],
 [1411, 1448],
 [1449, 1472],
 [1473, 1514],
 [1515, 1535],
 [1536, 1551],
 [1552, 1567],
 [1568, 1613],
 [1614, 1626],
 [1627, 1658],
 [1659, 1676],
 [1677, 1712],
 [1713, 1731],
 

In [37]:
# Find the amount of time that is skipped
skipped = df_preproc.loc[segments[1:, 0], 'Onset'].to_numpy() - df_preproc.loc[segments[:-1, 1], 'Offset'].to_numpy() 

np.round(skipped, 2)

array([0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.05, 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.01, 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.18, 0.  , 0.  , 0.  , 0.  , 0.04, 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.29])

In [ ]:


	# Create sequential pairs
	candidate_idxs = np.where(df_preproc['NWP_Candidate'].to_numpy())[0] # First get indices
	candidate_idxs = np.concatenate([[0], candidate_idxs], axis=0) # Add the first item for the first cut
	segments = np.vstack((candidate_idxs[:-1], candidate_idxs[1:]-1)).T # Stack and make pairs
	segment_indices = segments.tolist()
	
	# perform audio segmenting for the current subject
	out_fns, df_segments = cut_stimulus_segments(df_preproc, task=p.task, stim_fn=stim_fn, stim_out_dir=stim_out_dir, segment_indices=segment_indices, stim_type=p.stim_type)
	df_participant = create_participant_df(df_preproc, df_segments)

	# now write the file for jspsych to use
	jspsych_fn = os.path.join(jspsych_out_dir, f'{p.subject}_task-{p.task}')
	df_participant.to_csv(f'{jspsych_fn}.csv', index=False)
	df_participant.to_json(f'{jspsych_fn}.json', orient='records')